In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from neo.core import SpikeTrain
from quantities import ms, s, Hz
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram, instantaneous_rate
from elephant.kernels import GaussianKernel
from elephant.statistics import mean_firing_rate

import os 
import pandas as pd
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt

import sys
sys.path.append("G:/My Drive/WORKING_MEMORY/EXPERIMENTS/ELECTROPHYSIOLOGY/ANALYSIS/src/functions/")
import ephys_functions as ephys
import model_functions as mod
import behavioral_functions as beh

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings. filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

**Check the synch data from the paper**

In [ ]:
df = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/ANALYSIS_Figures/instantaneous firing rate_alltrial_5ms.csv", index_col=0)
df_final = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/WM_manuscript_FIGURES/Fig. 7. Synch/synch_data_trials_2beforeSti.csv", index_col=0)

In [ ]:
animal = "E22_2022-01-13_16-34-24.csv"
for animal in df_final.animal.unique():
    # file_name = 'synch_data_trials_2beforeSti'
    # df_final = pd.read_csv(file_name+'.csv', index_col=0)
    threshold = 0.5

    df_session = df_final.loc[df_final.animal==animal]

    fig, ax = plt.subplots(2,1 , figsize=(10,6))
    panel = ax[0]
    panel1 = ax[1]


    sns.lineplot(x="trial", y="synch_window",data=df_session, color='black',ci=68,ax=panel)      
    sns.lineplot(x="trial", y="synch",data=df_session, color='grey',ci=68,ax=panel1)      
    for plot in [panel, panel1]:
        plot.set_ylabel('Synch')
        plot.set_ylim(0.9,max(df_session.synch_window)+0.3)
        plot.fill_between(df_session['trial'],0.9 , 2.5, where=df_session['WM_roll'] <= threshold,
                    facecolor='indigo', alpha=0.3)
        plot.fill_between(df_session['trial'], 0.9, 2.5,  where=df_session['WM_roll'] >= threshold,
                        facecolor='darkgreen', alpha=0.3)
    panel.set_xlabel('Trials')
    panel.set_title('Mouse E22 13-01')
    plt.show()

**Re-do to obtain activity per bin for all neurons**

In [ ]:
path = r'E:\Ephys\summary_complete'

In [ ]:
# List of variables we want to recover
bin_size = 0.02
concat_df = pd.DataFrame()

for filename in os.listdir(path):
    print(filename)   

    if filename[-3:] != 'pdf':
        df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)
    else:
        continue
    
    # Add time before stimulus from the previous trial
    df = ephys.add_time_before_stimulus(df, 4)
    
    for align, start, stop, delay_epoch in zip(['END'],[-2],
                                               [0],[False]):

        # Sort per trial
        df = df.sort_values('trial')
        # Get unique neurons in the whole session
        unique_neurons = df.cluster_id.unique()
        
        #Create new aligment to the end of the session
        df['a_'+align] = df.fixed_times-df[align]

        # Define bin edges (e.g., 20 ms bins up to max time)
        bins = np.arange(-2, 0 + bin_size, bin_size)
        bin_labels = bins[:-1]  # Labels for bins (start times)

        # Assign each spike to a time bin
        df['a_END_bin'] = pd.cut(df['a_END'], bins=bins, labels=bin_labels, include_lowest=True)
        
        # Skip sessions with less than 30 neurons
        total_neurons_session = len(df.cluster_id.unique())
        if total_neurons_session < 30:
            continue
            
        for T in df.trial.unique(): 
            # Select the trial that we want to look at this time
            dft = df.loc[df.trial ==T]
            
            # Create a DataFrame with all possible combinations of cluster_id and a_END_bin
            all_combinations = pd.MultiIndex.from_product([unique_neurons, bin_labels], names=['cluster_id', 'a_END_bin']).to_frame(index=False)
        
            # Merge with the actual data to fill missing combinations with NaNs
            binned_spikes = pd.merge(all_combinations, dft.groupby(['cluster_id', 'a_END_bin']).size().reset_index(name='spike_count'), on=['cluster_id', 'a_END_bin'], how='left')

            # Fill NaNs in spike_count with 0
            binned_spikes['spike_count'].fillna(0, inplace=True)

            # Convert 'time_bin' back to numeric
            binned_spikes['a_END_bin'] = binned_spikes['a_END_bin'].astype(float)
            binned_spikes['trial'] = T
            binned_spikes['session'] = filename
            binned_spikes['WM_roll'] = dft.WM_roll.unique()[0]
            concat_df = pd.concat([concat_df, binned_spikes], axis=0)

E04_2021-03-30_11-20-16.csv
E04_2021-03-30_11-20-16complete_graph.pdf
E04_2021-04-03_16-12-15.csv
E04_2021-04-03_16-12-15complete_graph.pdf
E11_2021-05-12_12-56-16.csv
E11_2021-05-12_12-56-16complete_graph.pdf
E11_2021-05-13_12-34-49.csv
E11_2021-05-13_12-34-49complete_graph.pdf
E11_2021-05-14_14-27-21.csv
E11_2021-05-14_14-27-21complete_graph.pdf
E11_2021-05-24_14-33-29.csv
E11_2021-05-24_14-33-29complete_graph.pdf
E11_2021-05-26_12-20-58.csv
E11_2021-05-26_12-20-58complete_graph.pdf
E11_2021-05-27_12-15-58.csv


C:\Users\tiffany.ona\AppData\Local\Temp\ipykernel_27448\3232338482.py:9: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)


E11_2021-05-27_12-15-58complete_graph.pdf
E13_2021-05-25_16-26-57.csv


C:\Users\tiffany.ona\AppData\Local\Temp\ipykernel_27448\3232338482.py:9: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)


E13_2021-05-25_16-26-57complete_graph.pdf
E13_2021-05-26_15-01-42.csv
E13_2021-05-26_15-01-42complete_graph.pdf
E13_2021-06-06_11-29-34.csv
E13_2021-06-06_11-29-34complete_graph.pdf
E13_2021-06-08_11-22-23.csv
E13_2021-06-08_11-22-23complete_graph.pdf
E13_2021-06-09_12-14-21.csv
E13_2021-06-09_12-14-21complete_graph.pdf
E14_2021-03-29_13-01-12.csv
E14_2021-03-29_13-01-12complete_graph.pdf
E14_2021-03-31_13-07-18.csv
E14_2021-03-31_13-07-18complete_graph.pdf
E14_2021-04-02_12-53-42.csv


In [ ]:
df = concat_df.drop_duplicates(subset=['session', 'cluster_id', 'trial', 'WM'])
grp1 = df.groupby(['session', 'WM', 'cluster_id']).trial.nunique().reset_index()
grp2 = grp1.groupby(['session', 'WM']).agg(mean_trial = ('trial', 'mean'), std_trial = ('trial', 'std')).reset_index()
grp2

,session,WM,mean_trial,std_trial
0,E04_2021-03-30_11-20-16.csv,0,7.0,0.0
1,E04_2021-03-30_11-20-16.csv,1,184.0,0.0


In [ ]:
concat_df['WM'] = np.where(concat_df['WM_roll'] > 0.5, 1, 0)
concat_df.drop(columns=['WM_roll'], inplace=True)

In [ ]:
concat_df.to_csv(r'C:\Users\tiffany.ona\Documents\working_memory\data\3_11_Review_Prepare the firing rate for rank analysis\20ms_rank_df_fixed_V2.csv')

In [ ]:
concat_df = pd.read_csv(r'C:\Users\tiffany.ona\Documents\working_memory\data\3_11_Review_Prepare the firing rate for rank analysis\20ms_rank_df_fixed.csv', index_col=0)